# Classify the MNIST database using a Convolutional Neural Network

## Objectives

* 


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
tf.__version__

'0.10.0'

<img align="left" src="files/old.png"> **Old Code:** Load data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


<img align="left" src="files/old.png"> **Old Code:** Plot the result

In [4]:
def plot_img(img,size,channels,max_number): 
    
    fig, ax = plt.subplots(channels ,max_number, squeeze=False)

    for j in range(0,max_number):
        for i in range(0,channels):
            
            temp_img = img[j].transpose()         
            ax[i][j].imshow(temp_img[i].transpose().reshape((size, size)), cmap='gray', interpolation='nearest')
        
            ax[i][j].get_xaxis().set_visible(False)
            ax[i][j].get_yaxis().set_visible(False)

    plt.show()

<img align="left" src="files/old.png"> **Old Code:** Convolutional layer

In [5]:
def convolutional_layer(input, filter_size, in_channels, out_channels):

    filter_shape = [filter_size, filter_size, in_channels, out_channels]
    biases_shape = [out_channels]
    
    filter = tf.Variable(tf.truncated_normal(filter_shape))
    biases = tf.Variable(tf.truncated_normal(biases_shape))
    
    result = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME') + biases
    result = tf.nn.max_pool(value=result,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')
    result = tf.nn.relu(result)
    
    return result

## Build the computational graph using old functions

In [6]:
sess = tf.Session()

In [7]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [8]:
x_2d = tf.reshape(x, [-1, 28, 28, 1])

In [9]:
conv_1_out = convolutional_layer(x_2d, 5, 1, 16)

In [10]:
conv_1_out

<tf.Tensor 'Relu:0' shape=(?, 14, 14, 16) dtype=float32>

In [11]:
conv_2_out = convolutional_layer(conv_1_out, 5, 16, 36)

In [12]:
conv_2_out

<tf.Tensor 'Relu_1:0' shape=(?, 7, 7, 36) dtype=float32>

<img align="left" src="files/new.png"> **New Code**

In [13]:
def flatten_layer(input):
    
    layer_shape = input.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(input, [-1, num_features])
    
    return layer_flat, num_features

In [14]:
layer_flat, num_features = flatten_layer(conv_2_out)

<img align="left" src="files/new.png"> **New Code**

In [15]:
def fully_connected_layer(input, num_features, num_outputs):
    
    weights = tf.Variable(tf.truncated_normal([num_features, num_outputs], stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))

    output = tf.matmul(input, weights) + biases
    
    return output

In [16]:
layer_1 = fully_connected_layer(layer_flat, num_features, 128)

In [17]:
layer_1

<tf.Tensor 'add_2:0' shape=(?, 128) dtype=float32>

In [18]:
layer_1 = tf.nn.relu(layer_1)

In [19]:
layer_2 = fully_connected_layer(layer_1, 128, 10)

In [20]:
layer_2

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

In [21]:
y_pred = tf.nn.softmax(layer_2)

In [22]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [23]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_2,labels=y)

In [28]:
cost = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [25]:
sess.run(tf.initialize_all_variables())

In [26]:
for i in range(0,1000):
    x_batch, y_true_batch = data.train.next_batch(64)
    sess.run(train_step,feed_dict={x: x_batch,  y: y_true_batch})

In [27]:
y_pred_cls = tf.argmax(y_pred, dimension=1)
y_true_cls = tf.argmax(y, dimension=1)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print sess.run(accuracy, feed_dict={x: data.test.images, y: data.test.labels})

0.9332
